In [6]:
# Import necessary libraries

!pip install ultralytics -q
# !pip install gradio -q
# !pip install pandas -q
!pip install deepface -q

In [7]:
# Import necessary libraries
from ultralytics import YOLO
from ultralytics.engine.results import Results  
from deepface import DeepFace
from PIL import Image
# import gradio as gr
import shutil
import pandas
import cv2
import os

In [8]:
def faceRecognition(input_image):

    # Path to the directory containing cropped objects
    cropped_objects_dir = "./faces/"

    # Initialize a list to store the extracted names
    extracted_names = []
    
    #NO need to run this When database is fixed
    # file_to_delete = "./database/representations_facenet512.pkl"
    # if os.path.exists(file_to_delete):
    #     os.remove(file_to_delete)    

    # Iterate through the image files in the directory
    for filename in os.listdir(cropped_objects_dir):
        if filename.endswith(".jpg"):
            img_path = os.path.join(cropped_objects_dir, filename)
            model = DeepFace.find(img_path=img_path, db_path="database", enforce_detection=False, model_name="Facenet512")

            # Check if a face was recognized in the image
            if model and len(model[0]['identity']) > 0:
                # Extract the name and append it to the list
                name = model[0]['identity'][0].split('/')[1]
            else:
                # If no face is recognized, set name to 'unknown'
                name = 'unknown'

            extracted_names.append(name)

    return extracted_names

In [9]:
def faceExtraction(input_image, model, results):
    # Load the image
    image = Image.open(input_image)
    detected_objects = []

    if hasattr(results, 'boxes') and hasattr(results, 'names'):
        for box in results.boxes.xyxy:
            object_id = int(box[-1])
            object_name = results.names.get(object_id)
            x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])

            detected_objects.append((object_name, (x1, y1, x2, y2)))

    # Create or clear the 'faces' directory
    if os.path.exists("faces"):
        shutil.rmtree("faces")
    os.makedirs("faces")

    # Crop and save each detected object
    for i, (object_name, (x1, y1, x2, y2)) in enumerate(detected_objects):
        object_image = image.crop((x1, y1, x2, y2))
        object_image.save(f"faces/face{i}.jpg")
        
    return 0

In [10]:
def faceDetection(input_image):
    model = YOLO('best.pt')
    results: Results = model.predict(input_image)[0]

    return faceExtraction(input_image, model, results)

In [11]:
image = input("enter image path : ")
faceDetection(image)
names = faceRecognition(image)
print(names)

enter image path :  World-Leaders.jpg



image 1/1 D:\My\World-Leaders.jpg: 384x640 5 faces, 105.0ms
Speed: 3.0ms preprocess, 105.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


Finding representations: 100%|█████████████████████████████████████████████████████████| 20/20 [00:33<00:00,  1.66s/it]


23-12-25 19:36:34 - Representations stored in database/representations_facenet512.pkl file.Please delete this file when you add new identities in your database.
23-12-25 19:36:34 - find function lasts 33.70693802833557 seconds
23-12-25 19:36:34 - ⚠️ Representations for images in database folder were previously stored in representations_facenet512.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-25 19:36:34 - There are 20 representations found in representations_facenet512.pkl
23-12-25 19:36:35 - find function lasts 0.6045479774475098 seconds
23-12-25 19:36:35 - ⚠️ Representations for images in database folder were previously stored in representations_facenet512.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-25 19:36:35 - There are 20 representations found in representations_facenet512.pkl
23-12-25 19: